In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mtl
import autogluon as ag
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from autogluon.timeseries import TimeSeriesPredictor,TimeSeriesDataFrame
from mymetrics import ag_accuracy_scorer

In [3]:
df_all = pd.read_csv("Downloads/archive (3)/continuous dataset_timeseries.csv")
print("There are %0.0f" %df_all.shape[0] + " repeated measures and %0.0f" %df_all.shape[1] +" variables in the dataset" )
df_all.head()

There are 48048 repeated measures and 18 variables in the dataset


,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id
0,1/3/2015 0:00,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,1
1,1/3/2015 1:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,1
2,1/3/2015 2:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0,1
3,1/3/2015 3:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0,1
4,1/3/2015 4:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0,1


In [4]:
df_all['datetime'] = pd.to_datetime(df_all['datetime'],format="%m/%d/%Y %H:%M")

In [5]:
df_all.head()

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school,item_id
0,2015-01-03 00:00:00,960.3360,25.884526,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,1
1,2015-01-03 01:00:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0,1
2,2015-01-03 02:00:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0,1
3,2015-01-03 03:00:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0,1
4,2015-01-03 04:00:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0,1


In [6]:
df_all=df_all.drop(['QV2M_dav','QV2M_toc','W2M_dav','TQL_dav','QV2M_san','W2M_toc','TQL_toc','school','Holiday_ID'],axis=1)

In [7]:
df_all.head(2)

,datetime,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday,item_id
0,2015-01-03 00:00:00,960.336,25.884526,23.482446,0.001855,10.328949,22.662134,0,1
1,2015-01-03 01:00:00,970.345,25.865259,23.482446,0.001855,10.328949,22.662134,0,1


In [8]:
df_all.tail(2)

,datetime,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday,item_id
48046,2020-06-26 22:00:00,1081.568,26.971155,25.393030,0.144531,2.396369,23.658655,0,2002
48047,2020-06-26 23:00:00,1041.624,26.867487,25.258112,0.108063,2.720871,23.601862,0,2002


In [9]:
train_data = df_all[df_all['datetime'] < '2020-01-01']
test_data = df_all[df_all['datetime'] >= '2020-01-01']
print(train_data.shape,test_data.shape)

(43776, 9) (4272, 9)


In [10]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_data,
    id_column="item_id",
    timestamp_column="datetime"
)
train_data.tail()

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
1824    2019-12-31 19:00:00   1301.6065  26.635645  25.135645  0.064240   
        2019-12-31 20:00:00   1250.9634  26.495935  24.769373  0.058838   
        2019-12-31 21:00:00   1193.6802  26.354456  24.479456  0.038086   
        2019-12-31 22:00:00   1130.4575  26.166895  24.112207  0.020386   
        2019-12-31 23:00:00   1084.4737  25.976373  23.663873  0.019531   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
1824    2019-12-31 19:00:00  3.086798  23.620020        1  
        2019-12-31 20:00:00  3.659980  23.284998        1  
        2019-12-31 21:00:00  3.769294  23.041956        1  
        2019-12-31 22:00:00  3.872397  22.862207        1  
        2019-12-31 23:00:00  4.165276  22.726373        1

In [11]:
train_data1=train_data.iloc[:-24]
train_data1.tail()

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
1823    2019-12-30 19:00:00   1384.1848  26.609583  26.375208  0.183777   
        2019-12-30 20:00:00   1345.1400  26.413110  25.866235  0.181519   
        2019-12-30 21:00:00   1309.8852  26.242517  25.367517  0.174683   
        2019-12-30 22:00:00   1257.7586  26.094720  24.946283  0.162659   
        2019-12-30 23:00:00   1183.7685  25.976465  24.601465  0.150940   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
1823    2019-12-30 19:00:00  3.093210  23.945520        0  
        2019-12-30 20:00:00  3.180171  23.795923        0  
        2019-12-30 21:00:00  3.311305  23.687830        0  
        2019-12-30 22:00:00  3.428035  23.571283        0  
        2019-12-30 23:00:00  3.289671  23.413965        0

In [12]:
test=train_data.tail(24)
test

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
1824    2019-12-31 00:00:00   1144.1280  25.867090  24.343652  0.137695   
        2019-12-31 01:00:00   1090.5538  25.788416  24.124353  0.125000   
        2019-12-31 02:00:00   1061.3880  25.741418  23.991418  0.110718   
        2019-12-31 03:00:00   1025.7300  25.740228  24.021478  0.091614   
        2019-12-31 04:00:00   1016.7209  25.736047  23.970422  0.063751   
        2019-12-31 05:00:00   1015.7316  25.714441  23.698816  0.048325   
        2019-12-31 06:00:00   1049.8719  25.706934  23.433496  0.035583   
        2019-12-31 07:00:00   1099.1508  26.247644  24.060144  0.037125   
        2019-12-31 08:00:00   1194.5365  26.969812  24.891687  0.036758   
        2019-12-31 09:00:00   1241.3650  27.598352  26.160852  0.048843   
        2019-12-31 10:00:00   1321.3808  28.301416  27.223291  0.057846   
        2019-12-31 11:00:00   1343.1807  28.962671  28.040796  0.063354   
        2019-12-31 12:00:00   1292.4334  29.487726  28.581476  0.065247   
        2019-12-31 13:00:00   1264.0997  29.764886  28.858636  0.070465   
        2019-12-31 14:00:00   1272.5977  29.844080  28.859705  0.082886   
        2019-12-31 15:00:00   1250.3840  29.677972  28.568597  0.091949   
        2019-12-31 16:00:00   1223.4536  29.095850  27.970850  0.107483   
        2019-12-31 17:00:00   1193.3218  28.112024  26.893274  0.102570   
        2019-12-31 18:00:00   1287.3592  26.999292  25.733667  0.081604   
        2019-12-31 19:00:00   1301.6065  26.635645  25.135645  0.064240   
        2019-12-31 20:00:00   1250.9634  26.495935  24.769373  0.058838   
        2019-12-31 21:00:00   1193.6802  26.354456  24.479456  0.038086   
        2019-12-31 22:00:00   1130.4575  26.166895  24.112207  0.020386   
        2019-12-31 23:00:00   1084.4737  25.976373  23.663873  0.019531   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
1824    2019-12-31 00:00:00  3.224171  23.242090        1  
        2019-12-31 01:00:00  3.292790  23.054041        1  
        2019-12-31 02:00:00  3.770263  22.882043        1  
        2019-12-31 03:00:00  5.408209  22.732416        1  
        2019-12-31 04:00:00  7.466676  22.587610        1  
        2019-12-31 05:00:00  7.238291  22.456628        1  
        2019-12-31 06:00:00  5.045374  22.378809        1  
        2019-12-31 07:00:00  5.216480  22.630457        1  
        2019-12-31 08:00:00  6.040354  23.610437        1  
        2019-12-31 09:00:00  7.625834  24.746790        1  
        2019-12-31 10:00:00  8.642355  25.567041        1  
        2019-12-31 11:00:00  8.774728  26.134546        1  
        2019-12-31 12:00:00  8.378562  26.581476        1  
        2019-12-31 13:00:00  7.585188  26.811761        1  
        2019-12-31 14:00:00  6.753402  26.750330        1  
        2019-12-31 15:00:00  5.990585  26.396722        1  
        2019-12-31 16:00:00  5.503515  25.752100        1  
        2019-12-31 17:00:00  3.498447  24.916711        1  
        2019-12-31 18:00:00  2.659907  24.132104        1  
        2019-12-31 19:00:00  3.086798  23.620020        1  
        2019-12-31 20:00:00  3.659980  23.284998        1  
        2019-12-31 21:00:00  3.769294  23.041956        1  
        2019-12-31 22:00:00  3.872397  22.862207        1  
        2019-12-31 23:00:00  4.165276  22.726373        1

In [13]:
test_data = TimeSeriesDataFrame.from_data_frame(
    test_data,
    id_column="item_id",
    timestamp_column="datetime"
)
test_data.head()

nat_demand    T2M_toc    T2M_san   TQL_san  \
item_id timestamp                                                         
1825    2020-01-01 00:00:00   1032.2630  25.809259  23.301447  0.021866   
        2020-01-01 01:00:00   1017.5806  25.650842  23.041467  0.023796   
        2020-01-01 02:00:00   1000.2797  25.501489  22.806177  0.024910   
        2020-01-01 03:00:00    987.2383  25.430994  22.657556  0.025208   
        2020-01-01 04:00:00    970.3667  25.360132  22.539819  0.022789   

                              W2M_san    T2M_dav  holiday  
item_id timestamp                                          
1825    2020-01-01 00:00:00  4.559083  22.598322        1  
        2020-01-01 01:00:00  4.846412  22.471155        1  
        2020-01-01 02:00:00  5.204608  22.368677        1  
        2020-01-01 03:00:00  5.628076  22.282556        1  
        2020-01-01 04:00:00  5.729821  22.203882        1

In [14]:
WEEKEND_INDICES = [3]
timestamps = test_data.index.get_level_values("timestamp")
test_data["holiday"] = timestamps.weekday.isin(WEEKEND_INDICES).astype(float)

In [15]:
predictor=TimeSeriesPredictor(prediction_length=10,target='nat_demand').fit(train_data1,presets='fast_training',hyperparameters='default')

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20231222_015702'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          8
GPU Count:          0
Memory Avail:       2.07 GB / 7.86 GB (26.4%)
Disk Space Avail:   86.85 GB / 237.55 GB (36.6%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 10,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'nat_demand',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 43752 rows, 1823 time series. Median time series length is 24 (min=24, max=24). 

Provided dataset contains follo

In [17]:
predictions=predictor.predict(test)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [18]:
predictions=predictions.droplevel(level='item_id')
predictions

,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
timestamp,,,,,,,,,,
2020-01-01 00:00:00,1299.737244,1099.111817,1200.868607,1261.622132,1284.293289,1299.737244,1334.325135,1357.068482,1369.392868,1389.709656
2020-01-01 01:00:00,1295.452892,1087.881359,1165.350735,1224.498791,1268.230679,1295.452892,1317.977123,1328.605419,1351.772396,1375.156689
2020-01-01 02:00:00,1298.804544,1107.208719,1137.914972,1201.104410,1246.362055,1298.804544,1313.816080,1336.413004,1360.691614,1369.372721
2020-01-01 03:00:00,1267.758856,1064.010992,1114.827719,1175.073126,1229.254172,1267.758856,1305.942145,1324.064032,1335.726431,1358.391913
2020-01-01 04:00:00,1264.081993,1057.962242,1106.945167,1162.989708,1219.637367,1264.081993,1303.612205,1317.344017,1333.396492,1356.061973
2020-01-01 05:00:00,1283.273405,1079.523771,1118.061079,1176.765165,1240.144148,1283.273405,1303.356352,1320.972746,1333.140638,1355.806120
2020-01-01 06:00:00,1296.453135,1097.601999,1121.655603,1181.182322,1251.224756,1296.453135,1312.185740,1335.191538,1357.713312,1369.485849
2020-01-01 07:00:00,1309.197678,1110.346542,1134.400147,1193.926865,1263.969299,1309.197678,1324.930283,1347.936081,1370.457855,1382.230393
2020-01-01 08:00:00,1333.866393,1135.015258,1159.068862,1218.595580,1288.638015,1333.866393,1349.598998,1372.604797,1395.126571,1406.899108


In [19]:
y_true=test_data.head(10)
y_true.droplevel(level='item_id')

,nat_demand,T2M_toc,T2M_san,TQL_san,W2M_san,T2M_dav,holiday
timestamp,,,,,,,
2020-01-01 00:00:00,1032.2630,25.809259,23.301447,0.021866,4.559083,22.598322,0.0
2020-01-01 01:00:00,1017.5806,25.650842,23.041467,0.023796,4.846412,22.471155,0.0
2020-01-01 02:00:00,1000.2797,25.501489,22.806177,0.024910,5.204608,22.368677,0.0
2020-01-01 03:00:00,987.2383,25.430994,22.657556,0.025208,5.628076,22.282556,0.0
2020-01-01 04:00:00,970.3667,25.360132,22.539819,0.022789,5.729821,22.203882,0.0
2020-01-01 05:00:00,957.6767,25.316431,22.417993,0.016258,5.705475,22.144556,0.0
2020-01-01 06:00:00,922.3729,25.353662,22.447412,0.013000,5.860416,22.127100,0.0
2020-01-01 07:00:00,924.4692,26.073602,23.831415,0.015656,8.454858,22.417352,0.0
2020-01-01 08:00:00,954.3947,26.938074,25.461511,0.016876,9.816476,23.172449,0.0


In [20]:
a = (abs(y_true['nat_demand'] - predictions['mean']) / y_true['nat_demand']) * 100


In [21]:
p=pd.DataFrame(a,columns=['Percentage_Error'])
p

Percentage_Error
item_id timestamp                            
1825    2020-01-01 00:00:00         25.911443
        2020-01-01 01:00:00         27.307153
        2020-01-01 02:00:00         29.844137
        2020-01-01 03:00:00         28.414675
        2020-01-01 04:00:00         30.268484
        2020-01-01 05:00:00         33.998604
        2020-01-01 06:00:00         40.556291
        2020-01-01 07:00:00         41.616149
        2020-01-01 08:00:00         39.760457
        2020-01-01 09:00:00         36.499874